## Running a simulation of a System parametrized with a SMIRNOFF force field

This example shows how to use the Open Force Field Toolkit to create a parametrized `System` object that can be used to run a molecular dynamic simulation with OpenMM. If you want to run MD with a different engine, see the example in `examples/conversion_amber_gromacs/`.

### Create an OpenMM System

We start by loading a PDB file containing one copy of ethanol and cyclohexane. Our goal is to create an OFF `Topology` object describing this system that we can parametrize with the smirnoff99Frosst force field.

The two `Molecule` objects created from the SMILES strings can contain information such as partial charges and stereochemistry that is not included in an OpenMM topology. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the smirnoff99Frosst force field.

In [1]:
from simtk.openmm.app import PDBFile
from openforcefield.utils import get_data_file_path

We use the `get_data_file_path` utility function to easily access the data installed with the toolkit. Here you have the option to load example systems of increasing complexity. For speed, we recommend that you begin by loading a system with a single ethanol and a single cyclohexane.

In [2]:
# 1 molecule of ethanol and 1 of cyclohexane.
#pdb_file_path = get_data_file_path('systems/test_systems/1_cyclohexane_1_ethanol.pdb')

# 40%-60% cyclohexane-ethanol mixture.
#pdb_file_path = get_data_file_path('systems/packmol_boxes/cyclohexane_ethanol_0.4_0.6.pdb')
#pdb_file_path = get_data_file_path('systems/packmol_boxes/cyclohexane_water.pdb')
pdb_file_path = get_data_file_path('systems/packmol_boxes/ethanol_water.pdb')
#pdb_file_path = get_data_file_path('systems/packmol_boxes/cyclohexane.pdb')
#pdb_file_path = get_data_file_path('systems/packmol_boxes/cyclohexane_1_water.pdb')


pdbfile = PDBFile(pdb_file_path)

PDB files are not a reliable source of bond orders, so the toolkit requires users to supply a more detailed description of the molecule and its connectivity (currently either a SMILES, sdf, or mol2 file).

In [3]:
from openforcefield.topology import Molecule
ethanol = Molecule.from_smiles('CCO')
cyclohexane = Molecule.from_smiles('C1CCCCC1')
water = Molecule.from_smiles('O')
propane = Molecule.from_smiles('CCC')
methane = Molecule.from_smiles('C')
butanol = Molecule.from_smiles('CCCCO')

Alternatively, if you have `sdf` files of the molecules, or if you have OpenEye installed and `mol2` files available, you can get the same results as above by loading the detailed molecule information from the files.

In [4]:
from openforcefield.utils.toolkits import OpenEyeToolkitWrapper
if OpenEyeToolkitWrapper.is_available():
    ethanol = Molecule.from_file(get_data_file_path('molecules/ethanol.mol2'))
    cyclohexane = Molecule.from_file(get_data_file_path('molecules/cyclohexane.mol2'))

We now create the Open Force Field `Topology` describing the system from an OpenMM `Topology` object. The OFF `Topology` include more information (supplied by the two `Molecule` objects) than the OpenMM `Topology` such as (optionally) partial charges and stereochemistry. In this example, partial charges are not explicitly given, and `ForceField` will assign AM1/BCC charges as specified by the smirnoff99Frosst force field.

<div class="alert alert-block alert-info">
    <b>Note on partial charges:</b> The full 1.0.0 release will implement support for the definition of semiempirical partial charge treatment directly into the SMIRNOFF force field file (for details, see https://open-forcefield-toolkit.readthedocs.io/en/latest/smirnoff.html#partial-charge-and-electrostatics-models). Moreover, it will be possible to import charges directly from sdf and mol2 files.
</div>

In [4]:
from openforcefield.topology import Topology
from openforcefield.typing.engines.smirnoff import ForceField

# Create the Open Force Field Topology from an OpenMM Topology object.
omm_topology = pdbfile.topology
off_topology = Topology.from_openmm(omm_topology, unique_molecules=[ethanol, cyclohexane, water, propane, methane, butanol])

# Load the smirnof99Frosst force field.
forcefield = ForceField('smirnoff99Frosst.offxml')

# Parametrize the topology and create an OpenMM System.
system = forcefield.create_openmm_system(off_topology)

In [10]:
from simtk.openmm import XmlSerializer
from pprint import pprint
pprint(XmlSerializer.serialize(system))


('<?xml version="1.0" ?>\n'
 '<System openmmVersion="7.3" type="System" version="1">\n'
 '\t<PeriodicBoxVectors>\n'
 '\t\t<A x="3.7978000000000005" y="0" z="0"/>\n'
 '\t\t<B x="0" y="3.7978000000000005" z="0"/>\n'
 '\t\t<C x="0" y="0" z="3.7978000000000005"/>\n'
 '\t</PeriodicBoxVectors>\n'
 '\t<Particles>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="12.01078"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1

 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="1.007947"/>\n'
 '\t\t<Particle mass="15.99943"/>\n'
 

 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" 

 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".000220496

 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87

 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".0002204968" q=".39253002405166626" '
 'sig=".053453923088420355"/>\n'
 '\t\t\t\t<Particle eps=".87864" q="-.7850600481033325" '
 'sig=".2959921901149463"/>\n'
 '\t\t\t\t<Particle eps=".000220496

 '\t\t\t\t<Exception eps="0" p1="1380" p2="1381" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1380" p2="1382" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1381" p2="1382" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1383" p2="1384" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1383" p2="1385" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1384" p2="1385" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1386" p2="1387" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1386" p2="1388" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1387" p2="1388" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1389" p2="1390" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1389" p2="1391" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1390" p2="1391" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1392" p2="1393" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1392" p2="1394" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="1393" p2="1394"

 '\t\t\t\t<Exception eps="0" p1="3879" p2="3881" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3880" p2="3881" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3882" p2="3883" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3882" p2="3884" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3883" p2="3884" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3885" p2="3886" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3885" p2="3887" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3886" p2="3887" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3888" p2="3889" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3888" p2="3890" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3889" p2="3890" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3891" p2="3892" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3891" p2="3893" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3892" p2="3893" q="0" sig="1"/>\n'
 '\t\t\t\t<Exception eps="0" p1="3894" p2="3895"

 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3079" '
 'p2="3078" p3="3080"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3082" '
 'p2="3081" p3="3083"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3085" '
 'p2="3084" p3="3086"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3088" '
 'p2="3087" p3="3089"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3091" '
 'p2="3090" p3="3092"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3094" '
 'p2="3093" p3="3095"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3097" '
 'p2="3096" p3="3098"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3100" '
 'p2="3099" p3="3101"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3103" '
 'p2="3102" p3="3104"/>\n'
 '\t\t\t\t<Angle a="1.9722220547535925" k="418.40000000000003" p1="3106" '
 'p2="3

 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2679" p2="2681"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2682" p2="2683"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2682" p2="2684"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2685" p2="2686"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2685" p2="2687"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2688" p2="2689"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2688" p2="2690"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2691" p2="2692"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2691" p2="2693"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2694" p2="2695"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2694" p2="2696"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2697" p2="2698"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2697" p2="2699"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2700" p2="2701"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2700" p2="2702"/>\n'
 '\t\t\t\t<Bond d=".096" k="462750.4" p1="2703" p2="270

### Run a simulation

We can now use the `System` object to run molecular dynamics trajectories with OpenMM.

In [5]:
from simtk import openmm, unit

# Propagate the System with Langevin dynamics.
time_step = 1*unit.femtoseconds  # simulation timestep
temperature = 300*unit.kelvin  # simulation temperature
friction = 1/unit.picosecond  # collision rate
integrator = openmm.LangevinIntegrator(temperature, friction, time_step)

# Length of the simulation.
num_steps = 1000  # number of integration steps to run

# Logging options.
trj_freq = 1  # number of steps per written trajectory frame
data_freq = 1  # number of steps per written simulation statistics

# Set up an OpenMM simulation.
simulation = openmm.app.Simulation(omm_topology, system, integrator)

# Set the initial positions.
positions = pdbfile.getPositions() 
simulation.context.setPositions(positions)

# Randomize the velocities from a Boltzmann distribution at a given temperature.
simulation.context.setVelocitiesToTemperature(temperature)

# Configure the information in the output files.
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', trj_freq)
state_data_reporter = openmm.app.StateDataReporter('data.csv', data_freq, step=True,
                                                   potentialEnergy=True, temperature=True,
                                                   density=True)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [6]:
import time

print("Starting simulation")
start = time.clock()

# Run the simulation
simulation.step(num_steps)

end = time.clock()
print("Elapsed time %.2f seconds" % (end-start))
print("Done!")

Starting simulation
Elapsed time 58.20 seconds
Done!


If successful, the directory where your jupyter notebook is running should contain a `trajectory.pdb` file that you can visualize and a `data.csv` file including potential energy, density, and temperature of each frame.

In [13]:
(106.8/180) * 3.1415

1.8639566666666665

In [25]:
import networkx as nx
G = nx.Graph()
G.add_node(0, atomic_number = 6)
G.add_node(1, atomic_number = 6)
G.add_node(2, atomic_number = 6)
G.add_node(3, atomic_number = 1)
G.add_node(4, atomic_number = 1)
G.add_node(5, atomic_number = 1)
G.add_node(6, atomic_number = 1)
G.add_node(7, atomic_number = 1)
G.add_node(8, atomic_number = 1)
G.add_node(9, atomic_number = 8)
G.add_node(10, atomic_number = 8)
G.add_node(11, atomic_number = 8)
G.add_node(12, atomic_number = 8)
G.add_node(13, atomic_number = 6)
G.add_node(14, atomic_number = 58)
G.add_node(15, atomic_number = 80)
atoms = []
for idx in G.nodes:
    atoms.append(G.node[idx]['atomic_number'])
    

In [26]:
uniques = set(atoms)
counts_dict = dict([(unq, atoms.count(unq)) for unq in uniques])
print(counts_dict)
from simtk.openmm.app import Element
ele_counts = {}
if 6 in counts_dict:
    ele_counts['C'] = counts_dict[6]
    del counts_dict[6]
    
if 1 in counts_dict:
    ele_counts['H'] = counts_dict[1]
    del counts_dict[1]

sorted_atom_nums = sorted(counts_dict.keys())
for atom_num in sorted_atom_nums:
    ele_counts[Element.getByAtomicNumber(atom_num).symbol] = counts_dict[atom_num]

formula = ''
for ele, count in ele_counts.items():
    formula += f'{ele}{count}'
print(formula)

{1: 6, 6: 4, 8: 4, 80: 1, 58: 1}
C4H6O4Ce1Hg1
